# Bert Model 다운로드

In [ ]:
!wget https://storage.googleapis.com/bert_models/2018_11_23/multi_cased_L-12_H-768_A-12.zip

--2023-07-17 07:37:31--  https://storage.googleapis.com/bert_models/2018_11_23/multi_cased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.62.128, 172.253.115.128, 172.253.122.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.62.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 662903077 (632M) [application/zip]
Saving to: ‘multi_cased_L-12_H-768_A-12.zip’

multi_cased_L-12_H- 100%[===================>] 632.19M  77.5MB/s    in 8.1s    

2023-07-17 07:37:39 (77.8 MB/s) - ‘multi_cased_L-12_H-768_A-12.zip’ saved [662903077/662903077]



In [ ]:
import os

# 'bert'라는 이름의 디렉토리 만들기
if "bert" not in os.listdir():
    os.makedirs("bert")
else:
    pass

import zipfile
import shutil

# 만들어 둔 'bert' 디렉토리에 zip파일 압축해제
bert_zip = zipfile.ZipFile('multi_cased_L-12_H-768_A-12.zip')
bert_zip.extractall('bert')

# zip파일 닫기
bert_zip.close()

# 데이터셋 다운로드

### KorQuAD1.0 한국어 Q&A

In [ ]:
 !wget https://github.com/monologg/KoBERT-KorQuAD/blob/60898d3c0ed8fc724109cbe18855f9a48824216f/data/KorQuAD_v1.0_dev.json
 !wget https://github.com/monologg/KoBERT-KorQuAD/blob/60898d3c0ed8fc724109cbe18855f9a48824216f/data/KorQuAD_v1.0_train.json

--2023-07-17 07:37:48--  https://github.com/monologg/KoBERT-KorQuAD/blob/60898d3c0ed8fc724109cbe18855f9a48824216f/data/KorQuAD_v1.0_dev.json
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4778 (4.7K) [text/plain]
Saving to: ‘KorQuAD_v1.0_dev.json’

KorQuAD_v1.0_dev.js 100%[===================>]   4.67K  --.-KB/s    in 0s      

2023-07-17 07:37:48 (66.6 MB/s) - ‘KorQuAD_v1.0_dev.json’ saved [4778/4778]

--2023-07-17 07:37:49--  https://github.com/monologg/KoBERT-KorQuAD/blob/60898d3c0ed8fc724109cbe18855f9a48824216f/data/KorQuAD_v1.0_train.json
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4791 (4.7K) [text/plain]
Saving to: ‘KorQuAD_v1.0_train.json’

KorQuAD_v1.0_train. 100%[===================>]   4.68K  --.-KB/s    in 0s      

202

In [ ]:
# 현재 위치 폴더 내의 파일 list보기
os.listdir()

['.config',
 'bert',
 'KorQuAD_v1.0_train.json',
 'multi_cased_L-12_H-768_A-12.zip',
 'KorQuAD_v1.0_dev.json',
 'sample_data']

In [ ]:
# Google Drive와 Colab 연동
import os
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
path = "gdrive/My Drive/dp_project"

# 예측 모델을 만들기 위한 필요 라이브러리 호출

In [ ]:
# Colab의 기본 Tensorflow의 버전은 2.x이기 때문에 버전 지정
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

import pandas as pd # 데이터프레임 형식을 사용하기 위한 라이브러리
import numpy as np
import re
import pickle # 학습한 모델을 저장하거나, 열 떄 사용

import keras as keras
from keras.models import load_model # 모델 불러오기
from keras import backend as K
from keras import Input, Model
from keras import optimizers

from keras import backend as K
from keras.layers import Layer

import codecs
from tqdm import tqdm # 학습하는 동안 얼마나 학습했는지 눈으로 수치화해서 bar로 보여주는 라이브러리
import shutil
import json # 데이터 형식이 json 형식이므로 json 처리에 필요한 라이브러리

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
import warnings
warnings.filterwarnings(action='ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.logging.set_verbosity(tf.logging.ERROR)

## Bert 활용을 쉽게하기 위한 keras-bert 설치
## Adam optimizer 수정판 keras-radam 모듈 Import

In [ ]:
!pip install keras-bert
!pip install keras-radam

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for keras-bert: filename=keras_bert-0.89.0-py3-none-any.whl size=33501 sha256=6e91ee02b8a7c9193dd5031f3b0790c7827aed526dc1104f1b9c948db47fabfe
  Stored in directory: /root/.cache/pip/wheels/89/0c/04/646b6fdf6375911b42c8d540a8a3fda8d5d77634e5dcbe7b26
  Created wheel for keras-transformer: filename=keras_transformer-0.40.0-py3-none-any.whl size=12287 sha256=a48160ed250f6c2e7bfa72bd8013e11cbb89b2cb0b165068bc24ece8a65c1818
  Stored in directory: /root/.cache/pip/wheels/f2/cb/22/75a0ad376129177f7c95c0d91331a18f5368fd657f4035ba7c
  Created wheel for keras-embed-sim: filename=keras_embed_sim-0.10.0-py3-none-any.whl size=3943 sha256=fb739a553b6bccb6e3d1c28

# keras-bert 라이브러리에서 Bert Model 활용에 필요한 모듈 Import

In [ ]:
from keras_bert import load_trained_model_from_checkpoint, load_vocabulary
from keras_bert import Tokenizer
from keras_bert import AdamWarmup, calc_train_steps

from keras_radam import RAdam

In [ ]:
def korquad_json_to_dataframe_train(input_file_path, record_path = ['data','paragraphs','qas','answers'],
                           verbose = 1):
    if verbose:
        print("Reading the json file")
    file = json.loads(open(input_file_path).read())
    if verbose:
        print("processing...")
    # parsing different level's in the json file
    js = pd.io.json.json_normalize(file , record_path )
    m = pd.io.json.json_normalize(file, record_path[:-1] )
    r = pd.io.json.json_normalize(file,record_path[:-2])

    #combining it into single dataframe
    idx = np.repeat(r['context'].values, r.qas.str.len())
    ndx  = np.repeat(m['id'].values,m['answers'].str.len())
    m['context'] = idx
    js['q_idx'] = ndx
    main = pd.concat([ m[['id','question','context']].set_index('id'),js.set_index('q_idx')],1,sort=False).reset_index()
    main['c_id'] = main['context'].factorize()[0]
    if verbose:
        print("shape of the dataframe is {}".format(main.shape))
        print("Done")
    return main

In [ ]:
train = korquad_json_to_dataframe_train(path+"/KorQuAD_v1.0_train.json")

Reading the json file
processing...
shape of the dataframe is (60407, 6)
Done


In [42]:
train

,index,question,context,text,answer_start,c_id
0,6566495-0-0,바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?,1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로...,교향곡,54,0
1,6566495-0-1,바그너는 교향곡 작곡을 어디까지 쓴 뒤에 중단했는가?,1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로...,1악장,421,0
2,6566495-0-2,바그너가 파우스트 서곡을 쓸 때 어떤 곡의 영향을 받았는가?,1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로...,베토벤의 교향곡 9번,194,0
3,6566518-0-0,1839년 바그너가 교향곡의 소재로 쓰려고 했던 책은?,1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로...,파우스트,15,0
4,6566518-0-1,파우스트 서곡의 라단조 조성이 영향을 받은 베토벤의 곡은?,1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로...,합창교향곡,354,0
...,...,...,...,...,...,...
60402,6467478-1-1,뉴델리 메탈로 베타락마제가 처음 감염 된 지역은 어디인가?,"유전자의 이름은 인도의 수도 뉴델리의 이름을 따 붙여졌는데, 이는 2009년 용 (...",인도,73,9604
60403,6467478-2-0,균은 유전자를 균에게 전달 할 수있는데 이러한 현상을 나타낸 용어는 무엇인가?,"2010년 8월, 저널 The Lancet Infectious Diseases에 최...",유전자 전달,253,9605
60404,6467478-2-1,박테리아가 NDM-1 유전자를 가지고 있을때 발생하는 전파를 분석하기위해 사용된 영...,"2010년 8월, 저널 The Lancet Infectious Diseases에 최...",37건,129,9605
60405,6490801-2-0,NDM-1 유전자를 가진 박테리아가 감수성을 보인 폴리믹슨 계열 항생제는?,"2010년 8월, 저널 The Lancet Infectious Diseases에 최...",콜리스틴,404,9605


In [ ]:
# bert 훈련을 위한 사전 설정
# SEQ_LEN  - 문장의 최대 길이 기준
# SEQ_LEN보다 문장이 작을 경우 남은 부분은 0으로 채워짐
# SEQ_LEN보다 문장이 길경우 SEQ_LEN을 초과하는 부분이 잘려짐


In [47]:
!git init

Initialized empty Git repository in /content/.git/


In [52]:
!git config --global user.email "artyum15@kakao.com"
!git config --global user.name "chorimgenius"

In [54]:
!

fatal: pathspec 'Kor_QnA.ipynb' did not match any files
